In [ ]:
!pip install datasets
!pip install transformers
!pip install seacrowd>=0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sklearn.metrics.pairwise import cosine_similarity
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset

In [ ]:
class FloresMultiLangDataset(Dataset):
    def __init__(self, dataset, languages, tokenizer, max_length=512):
        """
        Custom PyTorch Dataset to load multiple languages from FLORES-200.
        Args:
            dataset (datasets.Dataset): The loaded dataset from FLORES-200.
            languages (dict): Dictionary of language names and their codes (e.g., {'English': 'eng_Latn', ...}).
            tokenizer (AutoTokenizer): The tokenizer for encoding text.
            max_length (int): Maximum sequence length for tokenization.
        """
        self.dataset = dataset
        self.languages = languages
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sentences = {}
        for lang_name, lang_code in self.languages.items():
            sentence = self.dataset[idx][f"sentence_{lang_code}"]
            sentences[lang_name] = sentence

            # # Tokenize the sentence
            # tokenized_input = self.tokenizer(
            #     template.format(sentence=sentence),
            #     return_tensors='pt',
            #     padding=True,
            #     truncation=True,
            #     max_length=self.max_length
            # )
            # tokenized_input = {k: v.to(device) for k, v in tokenized_input.items()}
            # sentences[f"{lang_name}_inputs"] = tokenized_input

        return sentences



def compare_languages(embeddings_dict, languages):
    """
    Compare each language embedding with every other language embedding and print cosine similarity scores.
    Args:
        embeddings_dict (dict): Dictionary of embeddings for each language.
        languages (dict): Dictionary of language names and codes.
    """
    lang_names = list(languages.keys())
    for i, lang1 in enumerate(lang_names):
        for lang2 in lang_names[i + 1:]:
            sim_score = cosine_similarity(embeddings_dict[lang1], embeddings_dict[lang2])
            print(f"Similarity between {lang1} and {lang2}: {sim_score[0][0]:.4f}")


In [ ]:
# Define the prompt template
template = 'This sentence: "{sentence}" means in one word:'

# Language dictionary mapping language names to their FLORES-200 codes
languages = {
    'English': 'eng_Latn',
    'Chinese_Simplified': 'zho_Hans',
    'Russian': 'rus_Cyrl',
    'Dutch': 'nld_Latn',
    'German': 'deu_Latn'
}

model_name_or_path='microsoft/Phi-3.5-mini-instruct'
batch_size=4
max_samples=10
avg_pooling=False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map='auto', output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

# Load FLORES-200 dataset
dataset = load_dataset('Muennighoff/flores200', 'all', split='devtest',trust_remote_code=True)

# Subset for faster testing
if max_samples:
    dataset = dataset.select(range(max_samples))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

flores200.py:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.01k [00:00<?, ?B/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Generating devtest split: 0 examples [00:00, ? examples/s]

In [ ]:
def collate_fn(batch):
    """
    Custom collate function for padding the inputs in a batch and including the actual sentences.
    """
    batch_dict = {}

    for lang_name in languages.keys():
        # Extract all the tokenized inputs, attention masks, and actual sentences for this language in the batch
        inputs = [item[f"{lang_name}_inputs"]['input_ids'].squeeze(0) for item in batch]
        attention_masks = [item[f"{lang_name}_inputs"]['attention_mask'].squeeze(0) for item in batch]
        sentences = [item[lang_name] for item in batch]  # Actual sentences

        # Pad the sequences for each language
        padded_inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
        padded_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True, padding_value=0)

        # Store padded inputs, masks, and sentences
        batch_dict[f"{lang_name}_inputs"] = {
            'input_ids': padded_inputs,
            'attention_mask': padded_masks
        }
        # batch_dict[f"{lang_name}_attention_masks"] = padded_masks
        batch_dict[lang_name] = sentences

    return batch_dict


In [ ]:
def get_embeddings_orginal(model, inputs, tokenizer, device,avg_pooling=False):
    embeddings = []
    for input in inputs:
        # Tokenize the sentence
        tokenized_input = tokenizer(
                template.format(sentence=input),
                return_tensors='pt',
                padding=True,
                truncation=True,
                max_length=512
        )
        input = {k: v.to(device) for k, v in tokenized_input.items()}
        # Get raw embeddings
        with torch.no_grad():
            # print(input)
            hidden_states = model(output_hidden_states=True, return_dict=True, **input).hidden_states
            if avg_pooling:
                last_layer = hidden_states[-1]
                attention_mask = input['attention_mask'].unsqueeze(-1).expand(last_layer.shape)
                outputs = (last_layer * attention_mask).mean(1)
            else:
                outputs = hidden_states[-1][:, -1, :]

            if outputs.dtype == torch.bfloat16:
                # bfloat16 not support for .numpy()
                outputs = outputs.float()

            embeddings.append(outputs.cpu().numpy())

    return np.vstack(embeddings)

In [ ]:
def get_embeddings(model, inputs, device, avg_pooling=False):
    """
    Get embeddings for a batch of inputs from the model.

    Args:
        model: The model to use for embedding extraction.
        inputs (dict): A dictionary containing tokenized inputs including input_ids and attention_mask.
        device: The device to move tensors to (CPU or GPU).
        avg_pooling (bool): Whether to use average pooling or not.

    Returns:
        numpy.ndarray: The embeddings for the input sentences.
    """

    # Get embeddings
    with torch.no_grad():
        # Compute hidden states for the entire batch
        hidden_states = model(**inputs, output_hidden_states=True, return_dict=True).hidden_states

        if avg_pooling:
            # Average pooling
            last_layer = hidden_states[-1]
            attention_mask = inputs['attention_mask'].unsqueeze(-1).expand(last_layer.shape)
            outputs = (last_layer * attention_mask).mean(1)
        else:
            # Last token embeddings
            outputs = hidden_states[-1][:, -1, :]

        if outputs.dtype == torch.bfloat16:
            # Convert bfloat16 to float
            outputs = outputs.float()

        return outputs.cpu().numpy()


In [ ]:
def find_most_similar(query_embedding, target_embeddings):
    similarities = cosine_similarity(query_embedding.reshape(1, -1), target_embeddings)[0]
    return np.argmax(similarities)

def compare_languages(embeddings_dict, languages):
    """
    Compare each language embedding with every other language embedding.
    Args:
        embeddings_dict (dict): Dictionary of embeddings for each language.
        languages (dict): Dictionary of language names and codes.

    Returns:
        list of dict: List containing cosine similarity scores between languages.
    """
    lang_names = list(languages.keys())
    results = []

    for i, lang1 in enumerate(lang_names):
        for lang2 in lang_names[i + 1:]:
            # sim_score = cosine_similarity(embeddings_dict[lang1], embeddings_dict[lang2])
            sim_score = find_most_similar(embeddings_dict[lang1], embeddings_dict[lang2])
            print("sim_score", sim_score)
            avg_score = sim_score  # Assuming the similarity score is a 2D array
            results.append({'Language 1': lang1, 'Language 2': lang2, 'Cosine Similarity': avg_score})

    return results

### With padding

In [ ]:
# Create dataset and dataloader
flores_dataset = FloresMultiLangDataset(dataset, languages, tokenizer)

data_loader = DataLoader(flores_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)
# Iterate through batches
print("\nEvaluating multi-language comparison...")
for batch in data_loader:
    embeddings_dict = {}

    # Get embeddings for each language
    for lang_name in languages.keys():
        inputs = batch[f"{lang_name}_inputs"]

        # inputs = batch[f"{lang_name}"]
        embeddings_dict[lang_name] = get_embeddings(model, inputs, device, avg_pooling)


    # Compare embeddings across all languages
    compare_languages(embeddings_dict, languages)


Evaluating multi-language comparison...


KeyError: 'English_inputs'

### Without padding

In [ ]:
data_loader = DataLoader(dataset, batch_size=2, shuffle=False)
# Iterate through batches
print("\nEvaluating multi-language comparison...")
for batch in data_loader:
    embeddings_dict = {}

    # Get embeddings for each language
    for lang_name in languages.keys():

        inputs = batch[f"{lang_name}"]
        embeddings_dict[lang_name] = get_embeddings_orginal(model, inputs, tokenizer, device, avg_pooling)

    # Compare embeddings across all languages
    compare_languages(embeddings_dict, languages)



Evaluating multi-language comparison...


KeyError: 'English'